In [1]:
import os
import requests
import pandas as pd
import time


In [39]:
kwords = pd.read_csv(os.getcwd().replace('scripts', 'data')+'/technical_buzzwords.csv')
words = pd.DataFrame(columns = ['topic', 'keywords'])
for i in range(kwords.shape[0]):
    words.loc[len(words.index)] = [kwords.iat[i,0], kwords.iat[i,1].split(", ")]

In [4]:
papers = pd.read_csv(os.getcwd().replace('scripts', 'data')+'/all_metadata.csv')

In [5]:
papers

,journal_id,volume_id,year,link,doi,date,pii,eid,scopus_id,citations,authors,affiliations,open_access
0,0,0,2023,https://www.sciencedirect.com/science/article/...,10.1016/j.hydres.2022.12.001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,2023,https://www.sciencedirect.com/science/article/...,10.1016/j.hydres.2022.12.002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,2023,https://www.sciencedirect.com/science/article/...,10.1016/j.hydres.2023.01.001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,2023,https://www.sciencedirect.com/science/article/...,10.1016/j.hydres.2023.01.002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,2023,https://www.sciencedirect.com/science/article/...,10.1016/j.hydres.2023.01.003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30675,28,68,2018,https://www.sciencedirect.com/science/article/...,10.1016/j.envsoft.2017.09.010,2018-01-01,S1364815216304005,2-s2.0-85032353211,8.503235e+10,54.0,"37111876100, 15069126500, 57189996027, 3574567...","60013141:60025182:60025182:60025182:60013141, ...",True
30676,28,68,2018,https://www.sciencedirect.com/science/article/...,10.1016/j.envsoft.2017.08.002,2018-01-01,S1364815216309550,2-s2.0-85032684995,8.503268e+10,5.0,"7006615627, 49661600900, 15132591800, 66033150...","60021439:60030486, 109682148:60021439:60021439...",True
30677,28,68,2018,https://www.sciencedirect.com/science/article/...,10.1016/j.envsoft.2017.09.011,2018-01-01,S136481521630843X,2-s2.0-85032736133,8.503274e+10,45.0,"56111768300, 37010006200, 43361846100, 5611302...",60068861:60020071:60068861:60024695:60102538:6...,True
30678,28,68,2018,https://www.sciencedirect.com/science/article/...,10.1016/j.envsoft.2017.09.002,2018-01-01,S136481521630648X,2-s2.0-85034039255,8.503404e+10,34.0,"52365181700, 22957372900, 7005864023, 89834056...","60032619, 60032619, 60032619:119771478:6003331...",True


In [8]:
word_contexts = pd.DataFrame(columns = ['paper_id', 'category', 'sub_word', 'context'])

In [20]:
keys = [
        "3cd3f5628a95bbd51760e0dc4acdff36",
        "d75a8debd1267fa990795879c18597ac",
        "5353196aa85c6f1cfc3a217184737814",
        "a0718a59c8b139814bf6c5563acb9fa9",
        "f19deff21c2efea371232594280b3540",
        "c57d9f7f8be8137d28a2f76625a3826b",
        "8649e480760588b2fa7f67427ae161db"
       ]

In [13]:
no_data = []

In [15]:
other_error = []

In [ ]:
keyi = 0

In [38]:
pd.DataFrame(columns = ['paper_index'], data = other_error).to_csv(os.getcwd().replace('scripts', 'data')+'/no_data.csv', index = False)



In [34]:
pd.DataFrame(columns = ['paper_index'], data = no_data)

,paper_index
0,0
1,1
2,2
3,3
4,4
...,...
370,29370
371,29371
372,30068
373,30240


In [22]:
word_contexts

,paper_id,category,sub_word,context


In [46]:
#get the full text
def fullText(i, keyi):
    doi = papers.iat[i,4]
#     print(doi)
    try:
        js = requests.get(
            f"https://api.elsevier.com/content/article/doi/{doi}",
            headers = {"X-ELS-APIKey":f"{keys[keyi]}", "Accept":"application/json"}

,
        )
        
        r = js.json()

        full_text = r['full-text-retrieval-response']['originalText']
#         full_text = re.sub(r'^https?:\/\/.[^\s]+', '', full_text)
#         print(full_text)
        
        abstr = r['full-text-retrieval-response']['coredata']['dc:description'][:25]
        
        index = full_text.find(abstr)
        if(index==-1):
            other.append(i)
            return None
        full_text = full_text[index:]
        
        full_text = ''.join(full_text.split('References')[:-1])
        
        

        full_text = full_text.lower().replace('/', ' ').replace('-', ' ').replace('\'', ' ').replace('_', ' ').replace('.', ' ').replace('(', '').replace(')', '').replace('*', '').replace(',', '')
    
#         print(full_text)
        
        return full_text
    except Exception as e:
        if js.status_code == 404:
            no_data.append(i)
        elif js.status_code ==429:
            keyi+=1
            if keyi==len(keys):
                keyi =0
        
        else:
            other.append(i)
        return None
    



In [50]:
for i in range(204,papers.shape[0]):
    print(i)
    ft = fullText(i, keyi)
    
    if ft == None:
        continue
#     print(ft)
    for j in range(words.shape[0]):
        subterms = words.iat[j, 1]
#         print(subterms)
        for found in find_keywords(subterms, ft):
            word_contexts.loc[len(word_contexts.index)] = [i, words.iat[j,0], found[0], found[1]]

10.1016/j.advwatres.2023.104474
serial jl 271718 291210 291707 291798 31 advances in water resources advancesinwaterresources 2023 05 27 2023 05 27 2023 06 07 2023 06 07 2023 06 23t21:35:40 1 s2 0 s0309170823001094 s0309 17082300109 4 s0309170823001094 10 1016 j advwatres 2023 104474 s250 s250 1 full text 1 s2 0 s0309170823x00086 2023 06 23t20:55:51 414136z 0 0 20230801 20230831 2023 2023 05 27t15:21:04 647017z articleinfo articlenumber articletitlenorm authfirstinitialnorm authfirstsurnamenorm cid cids contenttype copyright crossmark dateloaded dateloadedtxt datesearch datesort dateupdated dco docsubtype doctype doi eid ewtransactionid hubeid indexeddate issn issnnorm itemstage itemtransactionid itemweight openaccess openarchive pg pgfirst pii piinorm pubdateend pubdatestart pubdatetxt pubyr sortorder srctitle srctitlenorm srctype ssids alllist content subj subheadings suppl tomb volfirst volissue volumelist webpdf webpdfpagecount yearnav figure table body mmlmath acknowledge affil ar

In [52]:
word_contexts.iloc[68,3]

'rous media and implications for chemical reactions geophys  res  lett  42 2015 5316 5324 10 1002 2015gl064513 jimenez martinez j  anna p de tabuteau h  turuban r  borgne t le méheust y  2015  pore scale'

In [33]:
def find_keywords(k_list, abstract):
    ret = []
    
    old_abstract = abstract
    
    orig = old_abstract
    
    for word in k_list:
        word = word.lower().replace('-', ' ')
        curr_abstract = old_abstract
        c_orig = orig
        done = False
        while(True):
            if done:
                break
            try:
                found_i = curr_abstract.index(word)

                l_b = max(0, found_i - 100)
                u_b = min(len(curr_abstract), found_i+len(word)+100)
                
                if(l_b > u_b):
                    break
                try:
                    ret.append([word,c_orig[l_b:u_b]])
                except TypeError:
                    print(c_orig)
                    print(l_b + " " + u_b)
                
#                 print(word + " "+ curr_abstract[l_b:u_b])
                
                if(u_b == len(curr_abstract)):
                    break
                
                curr_abstract = curr_abstract[found_i + len(word):]
                c_orig = c_orig[found_i + len(word):]

            except ValueError:
                done = True
                continue
    
    
    return ret#returns the specific word & context